# Mapping 3
- Line of Interest (circle & straight line)
- Frequency spectrum mapping
- Linear interpolation
- Formant approach

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import Audio
import pyamapping

import numpy as np
from pya import *

%matplotlib widget

In [ ]:
s = startup()

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15) for x in range(n)] for y in range(n)])
potential = np.array([[simulation.parabola(x, y, n, offset=[0, 0], factor=10000) for x in range(n)] for y in range(n)])

multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

frames = simulation.sim(n, sim_fps, duration, slits, sim_speed, initial_state=initial_state, potential=potential)

In [ ]:
import matplotlib
plt.figure()
plt.pcolormesh(np.abs(frames[0]) ** 2, cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.square(np.abs(frames))), gamma=0.4))
# plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Sonification

In [ ]:
audio = np.abs(frames[::sim_fps//4].transpose(0, 2, 1).flatten())**2
sample_rate=128*128*4

In [ ]:
p_audio = Asig(audio, sr=sample_rate)
p_audio = p_audio.fade_in(0.005).fade_out(0.005).norm().stereo()

audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
p_audio.save_wavfile(audio_filename)
print(f"Sonification saved as {audio_filename}")

Audio(p_audio.mono().sig, rate=sample_rate)

In [ ]:
plt.figure()
p_audio.plot()
plt.show()

In [ ]:
plt.figure()
plt.scatter(x, y)
plt.show()

In [ ]:
fix, ax = plt.subplots()
p_audio.mono().to_stft(nperseg=4096).plot(ylim=(30, 20000), fn=pyamapping.ampdb, vmax=0, vmin=-120)
ax.set_yscale('log')

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, frame_amount, sim_fps, slits, n)

#plt.close()

In [ ]:
# combine audio & video
video.combine(audio_filename, video_filename)